# Assignment 4

Isaiah D. Murray (idm22@cornell.edu)

Daan van der Zwagg (dv239@cornell.edu)

#### In this assignment, we will conclude our analysis of whether the stop and frisk policy was racially discriminatory, but from a very different angle than our previous mapping analysis. We will rely heavily on logistic regression and regularized regression, as discussed in class. 

# Data processing (5 points)

First we need to do some data processing for consistency with previous of analysis of stop_and_frisk data. Load in the data "sqf_sample" and filter for stops between 2008 and 2012 (including both 2008 and 2012 in your sample). Filter for stops of white, Black, and Hispanic pedestrians using the suspect_race column. (5 points)


In [1]:
# importing necessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

In [2]:
# read in csv
sqf_sample = pd.read_csv("sqf_sample.csv")

In [3]:
data = sqf_sample

In [4]:
# check shape
print (data.shape)

(408615, 89)


In [5]:
# check columns
print (data.columns)

Index(['year', 'date', 'time', 'precinct', 'xcoord', 'ycoord', 'serial',
       'radio_run', 'inside_outside', 'location_housing', 'observation_period',
       'suspected_crime', 'stop_length', 'identification', 'reason_explained',
       'others_stopped', 'arrested', 'arrested_reason', 'summons_issued',
       'officer_uniform', 'officer_verbal', 'officer_shield', 'frisked',
       'searched', 'extra_reports', 'force_hands', 'force_wall',
       'force_ground', 'force_drawn', 'force_pointed', 'force_baton',
       'force_handcuffs', 'force_pepper', 'force_other', 'stopped_bc_object',
       'stopped_bc_desc', 'stopped_bc_casing', 'stopped_bc_lookout',
       'stopped_bc_clothing', 'stopped_bc_drugs', 'stopped_bc_furtive',
       'stopped_bc_violent', 'stopped_bc_bulge', 'stopped_bc_other',
       'frisked_bc_suspected_crime', 'frisked_bc_weapons', 'frisked_bc_attire',
       'frisked_bc_actual_crime', 'frisked_bc_noncompliance',
       'frisked_bc_threats', 'frisked_bc_prior', 'friske

In [6]:
# preview the data
data.head()

,year,date,time,precinct,xcoord,ycoord,serial,radio_run,inside_outside,location_housing,...,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,found_gun,found_weapon,id,lat,lon
0,2009,2009-02-13,13:01,9.0,990071.0,204546.0,730.0,False,False,neither,...,6.166667,185.0,black,brown,medium,False,False,1603997,40.728106,-73.978998
1,2010,2010-07-02,22:11,77.0,996834.0,184061.0,6369.0,False,False,neither,...,5.666667,175.0,black,brown,medium,False,False,2422905,40.671873,-73.954636
2,2007,2007-05-18,23:28,71.0,1002197.0,182674.0,2053.0,False,True,neither,...,5.750000,175.0,black,brown,medium,False,False,705704,40.668056,-73.935306
3,2010,2010-01-20,19:20,73.0,1010719.0,186857.0,967.0,False,True,transit,...,5.666667,180.0,black,brown,heavy,False,False,2133381,40.679516,-73.904570
4,2012,2012-01-18,19:50,48.0,1014014.0,249325.0,334.0,False,False,neither,...,6.000000,175.0,black,brown,medium,False,False,3425345,40.850964,-73.892414


In [7]:
# filtering for year
data = data[data["year"]>=2008]
data = data[data["year"]<=2012]

In [8]:
#checking the data shape
data.shape

(293278, 89)

In [9]:
# making sure that there are in fact only years 2008 to 2012
print (data["year"].unique())

[2009 2010 2012 2008 2011]


In [10]:
# seeing the unique values in suspect_race
print (data["suspect_race"].unique())

['black' 'hispanic' 'asian' 'white' nan 'other' 'native american']


In [11]:
# filtering for race
selected = ['black', 'hispanic', 'white']
data = data[data["suspect_race"].isin(selected)]

In [12]:
# seeing the unique values in suspect_race
print (data["suspect_race"].unique())

['black' 'hispanic' 'white']


In [13]:
data.shape

(273235, 89)

In [14]:
data.head()

,year,date,time,precinct,xcoord,ycoord,serial,radio_run,inside_outside,location_housing,...,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,found_gun,found_weapon,id,lat,lon
0,2009,2009-02-13,13:01,9.0,990071.0,204546.0,730.0,False,False,neither,...,6.166667,185.0,black,brown,medium,False,False,1603997,40.728106,-73.978998
1,2010,2010-07-02,22:11,77.0,996834.0,184061.0,6369.0,False,False,neither,...,5.666667,175.0,black,brown,medium,False,False,2422905,40.671873,-73.954636
3,2010,2010-01-20,19:20,73.0,1010719.0,186857.0,967.0,False,True,transit,...,5.666667,180.0,black,brown,heavy,False,False,2133381,40.679516,-73.904570
4,2012,2012-01-18,19:50,48.0,1014014.0,249325.0,334.0,False,False,neither,...,6.000000,175.0,black,brown,medium,False,False,3425345,40.850964,-73.892414
5,2010,2010-11-22,22:35,83.0,1008499.0,191844.0,8631.0,False,False,neither,...,5.750000,170.0,black,brown,medium,False,False,2650242,40.693211,-73.912556


## Using regression to analyze the frisk decision. (30 points)

We will start by testing for racial discrimination in the decision to conduct a frisk after a stop: ie, whether Black and Hispanic pedestrians are more likely to be frisked (patted down for weapons) after they are stopped, controlling for other factors.

a. Using statsmodels, perform a logistic regression, using `frisked` as the dependent variable and `suspect_race` as the independent variable, to assess how the probability of being frisked after a stop varies by race. Write a few sentences interpreting the results, making sure to answer the following questions: which value of suspect_race is omitted from the regression coefficients? Which race groups are most likely to be frisked after being stopped? How do you interpret the magnitude and sign of the coefficients? How do you interpret their statistical significance and confidence intervals? (5 points)

In [30]:
#preview the data
print (data['frisked_bc_weapons'])

0         False
1         False
2         False
3         False
4         False
          ...  
408610    False
408611    False
408612    False
408613     True
408614    False
Name: frisked_bc_weapons, Length: 408615, dtype: bool


In [32]:
# converting data to dummies
# edit d: 
dummies = pd.get_dummies(data['frisked_bc_weapons'])
print (dummies)

        False  True 
0           1      0
1           1      0
2           1      0
3           1      0
4           1      0
...       ...    ...
408610      1      0
408611      1      0
408612      1      0
408613      0      1
408614      1      0

[408615 rows x 2 columns]


In [35]:
# previewing the data
print (data['frisked_bc_weapons_dummies'].unique())
# 0 = False
# 1 = True

KeyError: 'frisked_bc_weapons_dummies'

In [41]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race)'
logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())

PatsyError: Error evaluating factor: NameError: name 'frisked_bc_weapons_dummies' is not defined
    frisked_bc_weapons_dummies ~ C(suspect_race)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^

b. Now perform a linear regression instead of a logistic regression using the same formula. How is the interpretation of the coefficients similar or different in the two regressions? What are the advantages of each? (5 points)

In [58]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race)'
lin_regress = sm.OLS(np.array(data["frisked_bc_weapons_dummies"]), np.array(data["suspect_race"])).fit()

print (lin_regress.summary())

#lin_reg=sm.OLS(ytrain,xtrain).fit()
#print(lin_reg.summary())

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

c. The regression using only race as an independent variable is a good starting point, but it does not control for any other variables. What other variables do you think are important to control for, and why?  (3 points)

d. Run a logistic regression where you control for both race and for the "precinct" variable, which encodes the police precinct in which the stop occurred. Make sure to control for precinct as a categorical, not a numerical, variable, by writing it as C(precinct) in the regression formula - why is this important to do?

How do the race coefficients change, and what does that mean? How does the interpretation of this regression differ from the regression in which you only control for race? Make one argument in favor of reporting results controlling for location, and one argument against it. (5 points)

In [45]:
#logistic regression
f = 'frisked_bc_weapons_dummies ~ C(suspect_race) + C(precinct)'

logitfit = smf.logit(formula = str(f), data = data).fit()
print(logitfit.summary())

Optimization terminated successfully.
         Current function value: 0.148051
         Iterations 9
                               Logit Regression Results                               
Dep. Variable:     frisked_bc_weapons_dummies   No. Observations:               273235
Model:                                  Logit   Df Residuals:                   273157
Method:                                   MLE   Df Model:                           77
Date:                        Thu, 04 Nov 2021   Pseudo R-squ.:                 0.01736
Time:                                16:12:30   Log-Likelihood:                -40453.
converged:                               True   LL-Null:                       -41167.
Covariance Type:                    nonrobust   LLR p-value:                6.634e-248
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept 

e. In a few sentences, explain why it is a BAD idea, conceptually, to control for the following variables if we are trying to assess whether the police racially discriminate in whom they frisk after a stop: a) "found.weapon", which encodes whether the frisk found a weapon, and b) "suspect.eye" and "suspect.hair", which encode the suspect's eye and hair color. (4 points)

f. In a few sentences, explain the problem of omitted variable bias in this analysis, and how it would undermine the conclusions. (4 points)

g. In a few sentences, explain why only examining whether someone is frisked after a stop might fail to provide a full picture of discrimination in the stop and frisk policy. (4 points)

## Outcome analysis using regularized regression (65 points)

Because of the issues with omitted variables in analyses like the one above, *outcome* tests are often used: these look not at the rate at which a decision is made (like the decision to frisk), but at the outcome of the decision (for example, if the frisk is conducted to find a weapon, does it actually find one?) Now we will use an outcome-style analysis. Specifically, we will fit a machine learning model to predict the probability that each stop which was conducted on suspicion the pedestrian possessed a weapon actually finds a weapon. Stops which are very unlikely to find a weapon arguably violate the Fourth Amendment, which prohibits unreasonable searches; if such stops disproportionately occur of certain race groups, the policy may violate the Fourteenth Amendment, which prohibits racial discrimination. 

a. In this portion of this analysis, you will be using a smaller version of the data to speed up model fitting. Load in "small_sqf_sample". As before, we need to do some data processing for consistency with previous of analysis of this data. Load in the data and filter for stops between 2008 and 2012 (including both 2008 and 2012); filter for stops of white, Black, and Hispanic pedestrians using the suspect_race column; and filter for stops conducted on suspicion of criminal posession of a weapon (ie, suspected_crime == 'cpw'). (5 points)

In [67]:
# load the data
sample_data = pd.read_csv("small_sqf_sample.csv")

In [68]:
# filtering for year between 2008 and 2012, renaming "sample_data" to "sample_data_filtered" 
sample_data_filtered = sample_data.loc[(sample_data["year"] >= 2008) & (sample_data["year"]<=2012)]

# check: print unique values in "year" columm
print(sample_data_filtered["year"].unique())

# filtering for race
selected_race = ['black', 'hispanic', 'white']
sample_data_filtered = sample_data_filtered[sample_data_filtered["suspect_race"].isin(selected_race)]

# check: print unique values in "suspect_race" column
print (sample_data_filtered["suspect_race"].unique())

# filtering for suspected crime
sample_data_filtered = sample_data_filtered[sample_data_filtered["suspected_crime"]=="cpw"]

# check: print unique values in "suspected_crime" column
print (sample_data_filtered["suspected_crime"].unique())

[2010 2012 2011 2008 2009]
['black' 'white' 'hispanic']
['cpw']


In [70]:
# check: display filtered dataset head
sample_data_filtered.head()

,year,date,time,precinct,xcoord,ycoord,serial,radio_run,inside_outside,location_housing,...,suspect_height,suspect_weight,suspect_hair,suspect_eye,suspect_build,found_gun,found_weapon,id,lat,lon
0,2010,2010-06-26,09:35,41.0,1011541.0,237353.0,3971.0,True,False,neither,...,6.000000,170.0,bald,brown,medium,False,False,2412904,40.818113,-73.901402
7,2008,2008-04-15,06:05,26.0,996993.0,236772.0,1310.0,True,False,housing,...,5.250000,130.0,brown,brown,thin,False,False,1150567,40.816551,-73.953962
17,2009,2009-03-10,05:15,79.0,999104.0,192913.0,3551.0,False,False,housing,...,5.583333,180.0,black,brown,medium,False,False,1651909,40.696166,-73.946433
20,2011,2011-01-18,18:55,100.0,1033623.0,152712.0,108.0,False,False,neither,...,6.333333,270.0,black,brown,heavy,False,False,2732153,40.585698,-73.822243
23,2010,2010-09-20,00:00,32.0,1000691.0,240368.0,7546.0,False,False,housing,...,5.500000,160.0,black,brown,medium,False,False,2539768,40.826415,-73.940594



b. We will be fitting the regression model 

`found_weapon ~ C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex + ADDITIONAL_CIRCUMSTANCE_COLUMNS` 

where ADDITIONAL_CIRCUMSTANCE_COLUMNS are any columns that begin with "stopped_bc" or "additional_" besides "additional_other" and "stopped_bc_other". You can get these columns by running

`ADDITIONAL_CIRCUMSTANCE_COLUMNS = [a for a in d.columns if ('stopped_bc' in a or 'additional_' in a) and a not in (['additional_other', 'stopped_bc_other'])]`

You should have 18 additional columns. These columns provide more information about the circumstances of the stop, and we include them for consistency with the original analysis and because they turn out to be important for predictive performance. 

Drop any rows with missing values in any of the variables you need. 

Now, we need to put the data into a format which sklearn can use later - ie, numpy arrays. Do this with "patsy" library and the "dmatrix" function. You can call dmatrix as follows:

`sqf_X = patsy.dmatrix('C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex +' + '+'.join(ADDITIONAL_CIRCUMSTANCE_COLUMNS),sqf_data, return_type='dataframe')`

and it will return a dataframe on which you can fit a regression model. The first argument to dmatrix is the formula that you want to use to make the dataframe; the second argument gives patsy the data you want to use; return_type='dataframe' ensures that you get a dataframe, not a patsy object which is hard to use.

Look at the output of dmatrix and explain what the columns mean. Why can't we just pass the columns from the original dataframe directly into the sklearn function? (8 points)


In [20]:
# create the additional columns (line breaks used for readibility)
ADDITIONAL_CIRCUMSTANCE_COLUMNS = [a for a in sample_data_filtered.columns 
                                   if ('stopped_bc' in a or 'additional_' in a) 
                                   and a not in (['additional_other', 'stopped_bc_other'])]

# check: length of additional columns → 18
print(len(ADDITIONAL_CIRCUMSTANCE_COLUMNS))

18


In [21]:
# set columns for model fitting; variables mentioned in regression model + additional cirumstances columns
model_columns = ["found_weapon", "precinct", "suspect_race", "location_housing", 
                 "year", "suspect_age", "suspect_height", "suspect_weight", "suspect_sex"]

model_columns = model_columns + ADDITIONAL_CIRCUMSTANCE_COLUMNS;

# check: print column names
print(model_columns)

['found_weapon', 'precinct', 'suspect_race', 'location_housing', 'year', 'suspect_age', 'suspect_height', 'suspect_weight', 'suspect_sex', 'stopped_bc_object', 'stopped_bc_desc', 'stopped_bc_casing', 'stopped_bc_lookout', 'stopped_bc_clothing', 'stopped_bc_drugs', 'stopped_bc_furtive', 'stopped_bc_violent', 'stopped_bc_bulge', 'additional_report', 'additional_investigation', 'additional_proximity', 'additional_evasive', 'additional_associating', 'additional_direction', 'additional_highcrime', 'additional_time', 'additional_sights']


In [22]:
# check: print amount columns
print(len(model_columns))

27


In [106]:
# filter data with selected model columns
sample_data_filtered = sample_data_filtered[model_columns]

#reset index
sample_data_filtered = sample_data_filtered.reset_index()

#check: display filtered dataset head with specific columns needed for the model
sample_data_filtered.head()

,index,found_weapon,precinct,suspect_race,location_housing,year,suspect_age,suspect_height,suspect_weight,suspect_sex,...,stopped_bc_bulge,additional_report,additional_investigation,additional_proximity,additional_evasive,additional_associating,additional_direction,additional_highcrime,additional_time,additional_sights
0,0,False,41.0,black,neither,2010,49,6.000000,170.0,male,...,False,False,False,False,False,False,False,False,False,False
1,1,False,26.0,hispanic,housing,2008,35,5.250000,130.0,female,...,False,False,True,True,False,False,False,False,False,False
2,2,False,79.0,black,housing,2009,30,5.583333,180.0,male,...,True,False,False,False,False,False,False,True,True,False
3,3,False,100.0,black,neither,2011,31,6.333333,270.0,male,...,False,True,False,False,False,False,False,True,True,False
4,4,False,32.0,hispanic,housing,2010,16,5.500000,160.0,male,...,False,False,False,False,False,False,False,True,False,False


In [107]:
# drop missing values we need in dataset
sample_data_dropped = sample_data_filtered.dropna()

# check: compare initial size of data and after drop
print("initial size:", sample_data_filtered.shape, "size after drop:", sample_data_dropped.shape)

initial size: (36271, 28) size after drop: (36056, 28)


In [108]:
# import patsy and dmatrix
import patsy
from patsy import dmatrices, dmatrix, demo_data

# set data in format sklearn can use with patsy and 
sqf_X = patsy.dmatrix('C(precinct) * C(suspect_race) + C(location_housing) + C(year) + suspect_age + suspect_height + suspect_weight + suspect_sex +' + '+'.join(ADDITIONAL_CIRCUMSTANCE_COLUMNS), sample_data_dropped, return_type='dataframe')


# check: see head dataframe converted to dmatrix
#print(sqf_X.columns)

for x in sqf_X.columns: print (x)

Intercept
C(precinct)[T.5.0]
C(precinct)[T.6.0]
C(precinct)[T.7.0]
C(precinct)[T.9.0]
C(precinct)[T.10.0]
C(precinct)[T.13.0]
C(precinct)[T.14.0]
C(precinct)[T.17.0]
C(precinct)[T.18.0]
C(precinct)[T.19.0]
C(precinct)[T.20.0]
C(precinct)[T.22.0]
C(precinct)[T.23.0]
C(precinct)[T.24.0]
C(precinct)[T.25.0]
C(precinct)[T.26.0]
C(precinct)[T.28.0]
C(precinct)[T.30.0]
C(precinct)[T.32.0]
C(precinct)[T.33.0]
C(precinct)[T.34.0]
C(precinct)[T.40.0]
C(precinct)[T.41.0]
C(precinct)[T.42.0]
C(precinct)[T.43.0]
C(precinct)[T.44.0]
C(precinct)[T.45.0]
C(precinct)[T.46.0]
C(precinct)[T.47.0]
C(precinct)[T.48.0]
C(precinct)[T.49.0]
C(precinct)[T.50.0]
C(precinct)[T.52.0]
C(precinct)[T.60.0]
C(precinct)[T.61.0]
C(precinct)[T.62.0]
C(precinct)[T.63.0]
C(precinct)[T.66.0]
C(precinct)[T.67.0]
C(precinct)[T.68.0]
C(precinct)[T.69.0]
C(precinct)[T.70.0]
C(precinct)[T.71.0]
C(precinct)[T.72.0]
C(precinct)[T.73.0]
C(precinct)[T.75.0]
C(precinct)[T.76.0]
C(precinct)[T.77.0]
C(precinct)[T.78.0]
C(precinct)[T.

## Analyze: dmatrix and differences

Look at the output of dmatrix and explain what the columns mean. Why can't we just pass the columns from the original dataframe directly into the sklearn function? (8 points)

* coding categorical data (TRUE/FALSE) to numerical data, by adding columns for each unique entry in a categorical column. It brings a 

* regression can work with numbers, not categorical features as strings

c. As discussed in class, when fitting machine learning models, you should always divide the dataset into a train, val, and test set. Randomly divide the filtered, processed data into three pieces - the train set (60%), the val set (20%) and the test set (20%). (5 points)

In [114]:
# divide datasets in train, val and test sets.
# source: https://stackoverflow.com/questions/17412439/how-to-split-data-into-trainset-and-testset-randomly

from sklearn.model_selection import train_test_split

# split randomnly initial data into 60/40
sqf_X_train, sqf_X_other = train_test_split(sqf_X, test_size=0.4)

# split other data into 50/50, therefor 40 (sqf_X_other) goes 20/20
sqf_X_val, sqf_X_test = train_test_split(sqf_X_other, test_size=0.5)

# critique: above method of randomizing twice can raise questions of true randomization (probability) of a single entry ending up in a specific set

# check: data lengths to verify 60/20/20

print("total:", len(sqf_X))
print("train:", len(sqf_X_train), ", train frac:", len(sqf_X_train) / len(sqf_X))
print("val:", len(sqf_X_val), ", val frac:", len(sqf_X_val) / len(sqf_X))
print("test:", len(sqf_X_test), ", test frac:", len(sqf_X_test) / len(sqf_X))


total: 36056
train: 21633 , train frac: 0.5999833592189927
val: 7211 , val frac: 0.19999445307299757
test: 7212 , test frac: 0.20002218770800975


d. We will be training a regularized logistic regression model to predict the outcome. When using many machine learning models, including regularized logistic regression, it is important to preprocess the input features so they are all on the same scale, for reasons discussed in class. In this case, we will take each column in the input data, subtract its mean, and divide by its standard deviation. This makes it so each column of the data has mean 0 and standard deviation 1. 

When scaling the data, it is important to compute the scaling using only the train set, as shown in class. The reason is that we are pretending that the train data is all we have access to to fit our model fitting pipeline, so we cannot "peek" at the validation or test sets to generate our scaling. Use sklearn's StandardScaler (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to fit a scaling transform on the train set and transform the train set (you can use the fit_transform method on the train set).Then apply the fitted StandardScaler model to the validation and test sets as well (using the transform --- not the fit --- method). (Look at the notebook we went through in class on regularization and lasso if you are confused.) The transformed datasets are the final datasets you will feed into your logistic regression model. (5 points)

In [120]:
# fit a scaling transform on the train set
# source: lecture_7_regularization_and_lasso.ipynb by Emma Pierson 

# Use standardscaler to normalize the columns of our matrix

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# fit_transform the trainset
sqf_X_train = scaler.fit_transform(sqf_X_train)

# check: print std and mean of train set
print(sqf_X_train.std(axis=1))
print(sqf_X_train.mean(axis=1))

[0.37971209 0.43240281 0.55916257 ... 0.45186899 0.95873626 0.48673457]
[-0.05046986 -0.07091865 -0.03522701 ... -0.0388898   0.01699454
 -0.06090443]


##### apply fitted standard scaler to validation set, using the transform method

sqf_X_val = scaler.transform(sqf_X_val)

# check: print std and mean of validation set
print(sqf_X_val.std(axis=0))
print(sqf_X_val.mean(axis=0))

In [57]:
# apply fitted standard scaler to test set, using the transform method

sqf_X_test = scaler.transform(sqf_X_test)

# check: print std and mean of validation set
print(sqf_X_test.std(axis=0))
print(sqf_X_test.mean(axis=0))

[0.00000000e+00 9.99930804e-01 9.53462782e-01 9.90587790e-01
 8.57971126e-01 1.20977547e+00 1.31854326e+00 1.03114594e+00
 1.22449009e+00 7.54682677e-01 8.84795371e-01 1.09516640e+00
 9.48639487e-01 9.94056121e-01 9.67272054e-01 1.03255349e+00
 1.11784985e+00 9.65485421e-01 9.15212265e-01 1.01615483e+00
 9.17222392e-01 1.01660544e+00 1.02398324e+00 1.05360179e+00
 1.03802377e+00 1.04399908e+00 1.02475101e+00 1.04289518e+00
 9.74692194e-01 1.01900602e+00 9.05196346e-01 1.02171418e+00
 1.03014314e+00 9.66095504e-01 1.04370879e+00 9.48860206e-01
 9.99930813e-01 1.29060090e+00 1.04432111e+00 1.02041373e+00
 1.35344281e+00 9.36237249e-01 1.01466581e+00 9.49801004e-01
 9.28111400e-01 1.01400312e+00 9.91383945e-01 8.32688567e-01
 9.62890600e-01 1.10371483e+00 1.02177618e+00 1.02029397e+00
 9.70876357e-01 1.02547919e+00 1.04675043e+00 9.74431480e-01
 1.02287335e+00 1.06014290e+00 1.02384508e+00 7.75081815e-01
 9.66938668e-01 9.67447657e-01 8.91488618e-01 8.10656937e-01
 9.32954995e-01 8.300817

e. Uisng sklearn.linear_model.LogisticRegression, fit a model on the train set to predict found_weapon. Set the "penalty" argument to "none" so that the model will not use any regularization; this corresponds to fitting a regular logistic regression model. If you get a `ConvergenceWarning`, increase the number of iterations using the `max_iter` argument; this means the model optimization needs more iterations to converge. (Look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) if you are uncertain which arguments to use!)

In [66]:
# import logistic regression model
from sklearn.linear_model import LogisticRegression

X = StandardScaler().fit_transform(X)
y = ??

# HELP: how to implement logistic regression and predict "found_weapon" -> what is Y?

# use logistic regression to fit a model on the train set to predit found_weapon
sqf_X_train_lr = LogisticRegression(penalty="none").fit(X,y) #sqf_X_train["predict_weapon"]


print(sqf_X_train_lr)

# predict found_weapon

#sqf_X_train_lr.predict(X[:2, :])

NameError: name 'X' is not defined

A standard measure of predictive performance for binary outcome variables like found_weapon is AUC. Higher values of AUC are better; an AUC of 1 means that the model is perfectly predicting the outcome; an AUC of 0.5 means that it is predicting it only as well as random chance. Report the AUC of the fitted model (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) on both the train and validation sets. What is the problem with using accuracy as a metric for this task? How does the train set AUC differ from the validation AUC, and does this make sense? (7 points)

## Analyze: AUC

f. Our logistic regression model is not using any regularization and appears to be overfitting. We will try to use regularization to reduce overfitting. You will be fitting an L1-penalized logistic regression model using code that looks something like

`LogisticRegression(C=sparsity_param, penalty='l1', solver='liblinear')`

(the "l1" specifies that we're using an L1 penalty, as discussed in class; the "liblinear" solver is an optimizer that works with the L1 penalty. See the logistic regression documentation for more details.) 

Increase and decrease the amount of regularization using different values of the C parameter, searching logarithmically over at least 20 values in the range from 1e-2 to 1. (Note: we do not recommend defining a variable named "C" in your code, because this may cause weird patsy issues, since patsy also uses "C". Give the variable another name. Sorry! I complained to the patsy people.) 

Print out the train set, val set, and test set AUC for each value of of the regularization parameter. Make a plot where the x-axis is the regularization parameter and the y-axis is AUC, with one line for train AUC, one line for val AUC, and one line for test AUC (use plt.semilogx to plot the lines so the x-axis will be logarithmic, making it easier to see the plot). Comment on the trends. Do you see evidence of overfitting? Explain. For the rest of this assignment, use the model with the highest AUC on the validation set. (10 points)

In a full analysis, it would make sense to play with other aspects of the model as well: for example, you could try using other forms of regularization (like L1 vs L2) or other classification algorithms besides logistic regression. The basic pattern, though, would be the same: fit the model on the train set, choose models on the val set, and once you've chosen your best model, assess your results (once!) on the test set.

g. Assess model *calibration* on the test set. This checks whether the model's predicted probabilities line up with the true probabilities, and is important to assess here because we will be analyzing the model's predicted probabilities. To assess calibration, take the 10% of rows of the test set with the highest model predictions and compare the mean model predicted probability (ie, the output of `predict_proba`) to the actual mean of the outcome variable. Repeat for the next 10% of rows, and for all 10% groups. Make a plot comparing the model predicted probabilities on each 10% group to the actual outcomes (mean predicted probability on the x-axis and mean actual outcome on the y-axis). You should end up with a plot with 10 points, one for each 10% group. Plot the line y=x so you can see how well the predicted probabilities line up with the actual probabilities - ideally, your points will lie on the line or close to it. (10 points)

Calibration is an issue for many machine learning models, including deep learning models, so this is always a good thing to check. In a full analysis, it would make sense to check calibration (and AUC) for subgroups as well (eg, each race group).

h. Using the test set, compute the fraction of observations which have lower than a 2% model-predicted probability of finding a weapon for Black, Hispanic, and white pedestrians. Stops below this threshold are extremely unlikely to have resulted in finding a weapon, arguably violating the Fourth Amendment. Repeat this for 20 thresholds evenly spaced between 1% and 5%. Make a graph where the x-axis is the threshold, and the y-axis is the fraction of stops falling below that threshold, with one line for each race group. Explain what you observe. Do you think this graph provides evidence for Fourteenth Amendment violations (racial discrimination)? (10 points)

i. The analysis you have performed in the second part of this assignment is very similar to the analysis in [this paper](https://5harad.com/papers/stop-and-frisk.pdf). Read the paper and write a few sentences about their main conclusions. (5 points)